In [ ]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras.models import load_model

In [ ]:
# Load the model from the .h5 file
model = load_model('model_NN_final.h5')

In [ ]:
file_path5 = 'data_parsing/Parsed_Data3.csv'
data3 = pd.read_csv(file_path5)

In [ ]:
# Apply the function to each row
data3['specific_sequence'] = data3.apply(extract_specific_sequence, axis=1)


data3['1st_pos'] = data3['specific_sequence'].apply(lambda x: x[0])
data3['2nd_pos'] = data3['specific_sequence'].apply(lambda x: x[1])
data3['3rd_pos'] = data3['specific_sequence'].apply(lambda x: x[2])
data3['4th_pos'] = data3['specific_sequence'].apply(lambda x: x[3])
data3['5th_pos'] = data3['specific_sequence'].apply(lambda x: x[4])

data3.head()

In [ ]:
seq_col = [col for col in data3.columns.tolist() if col.startswith('sequence_')]
data3.drop(seq_col, axis=1, inplace=True)
data3.drop('specific_sequence', axis=1, inplace=True)
data3['1st_pos'] = data3['1st_pos'].astype('category')
data3['2nd_pos'] = data3['2nd_pos'].astype('category')
data3['3rd_pos'] = data3['3rd_pos'].astype('category')
data3['4th_pos'] = data3['4th_pos'].astype('category')
data3['5th_pos'] = data3['5th_pos'].astype('category')

In [ ]:
grouped_data3 = data3.groupby("transcript_id")

# Apply the standardization function to the "0 mean" column within each group
data3["0 mean standardized"] = grouped_data3["0 mean"].transform(standardize_column_mean)
data3["0 sd standardized"] = grouped_data3["0 sd"].transform(standardize_column_sd)
data3["0 length standardized"] = grouped_data3["0 length"].transform(standardize_column_mean)

In [ ]:
final_predict_dataset3 = data3.drop(columns=['transcript_id','position','0 length', '0 sd', '0 mean', '0 min', '0 max'])

# Perform one-hot encoding for categorical columns
final_predict_dataset3 = pd.get_dummies(final_predict_dataset3, columns=['1st_pos', '2nd_pos', '3rd_pos', '4th_pos', '5th_pos'])


In [ ]:
transcript3 = data3[['transcript_id','position']]
data3_pred_prob = model.predict(final_predict_dataset3)

In [ ]:
data3_pred_p = pd.DataFrame(pd.DataFrame(data3_pred_prob))
data3_pred_p.columns = ['Score']
result_data3 = pd.merge(transcript3, data3_pred_p, left_index=True, right_index=True, how='inner')
agg_functions = {'Score': 'mean'}

result_data3 = result_data3.groupby(['transcript_id', 'position']).agg(agg_functions).reset_index()

In [ ]:
result_data3.to_csv('prediction_generation/Final_Data3_Result.csv', index=False)